## Flowers classifier using Transfer Learning and tf.data


Accuracy : 0.9090909090909091

Classification Report
              precision    recall  f1-score   support

           0    0.96429   0.90000   0.93103        60
           1    0.88750   0.98611   0.93421        72
           2    0.81538   0.89831   0.85484        59
           3    0.98462   0.86486   0.92086        74
           4    0.90698   0.89655   0.90173        87



In [0]:
#"""
# Google Collab specific stuff....
from google.colab import drive
drive.mount('/content/drive')

import os
!ls "/content/drive/My Drive"

USING_COLLAB = True
%tensorflow_version 2.x
#"""

In [0]:
# Setup sys.path to find MachineLearning lib directory

try: USING_COLLAB
except NameError: USING_COLLAB = False

%load_ext autoreload
%autoreload 2

import sys
if "MachineLearning" in sys.path[0]:
    pass
else:
    print(sys.path)
    if USING_COLLAB:
        sys.path.insert(0, '/content/drive/My Drive/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    else:
        sys.path.insert(0, '/Users/john/Documents/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    
    print(sys.path)

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys, random, warnings, time, copy, csv, gc
import numpy as np 

import IPython.display as display
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from tqdm import tqdm_notebook, tnrange, tqdm
import pandas as pd

import tensorflow as tf
print(tf.__version__)

AUTOTUNE = tf.data.experimental.AUTOTUNE
print("AUTOTUNE: ", AUTOTUNE)

from TrainingUtils import *

#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)

## Examine and understand data


In [0]:
# GLOBALS/CONFIG ITEMS

# Set root directory path to data
if USING_COLLAB:
    ROOT_PATH = "/content/drive/My Drive/ImageData/Flowers"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
else:
    ROOT_PATH = "/Users/john/Documents/ImageData/Flowers"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
        
# Establish global dictionary
parms = GlobalParms(ROOT_PATH=ROOT_PATH,
                    TRAIN_DIR="train", 
                    SMALL_RUN=False,
                    NUM_CLASSES=5,
                    IMAGE_ROWS=224,
                    IMAGE_COLS=224,
                    IMAGE_CHANNELS=3,
                    BATCH_SIZE=32,
                    EPOCS=10,
                    IMAGE_EXT=".jpg",
                    FINAL_ACTIVATION='sigmoid',
                    LOSS='binary_crossentropy',
                    METRICS=['accuracy'])

parms.print_contents()

In [0]:
#"""
# If not loaded, uncomment one of these to load the database as needed

# This loads the files into a temporary directory                                         
load_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)

# This loads the files into a actual directory, WILL TAKE LONGER TO UNZIP AND TRAIN.  But stored on Drive
#load_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
#                                         fname='flower_photos', untar=True, cache_subdir=parms.TRAIN_PATH)


# set new value for TRAIN_PATH
parms.set_train_path(load_dir) # If we downloaded the images, then overide TRAIN_PATH
print(load_dir, parms.TRAIN_PATH)

#"""

In [0]:
if parms.SMALL_RUN:
    max_subdir_files = 10
else:
    max_subdir_files = 1000000
    
images_list, sub_directories = load_file_names_labeled_subdir_Util(parms.TRAIN_PATH, 
                                                                   parms.IMAGE_EXT, 
                                                                   max_dir_files=max_subdir_files)

images_list_len = len(images_list)
print("Number of images: ", images_list_len)

random.shuffle(images_list) # randomize the list

# Set the class names.
parms.set_class_names(sub_directories)
print("Classes: ", parms.NUM_CLASSES, 
      "   Labels: ", len(parms.CLASS_NAMES), 
      "  ", parms.CLASS_NAMES)


In [0]:
# Show a few images
for image_path in images_list[:3]:
    print(image_path)
    display.display(Image.open(str(image_path)))

In [0]:
# Create Dataset from list of images
full_dataset = tf.data.Dataset.from_tensor_slices(np.array(images_list))
full_dataset = full_dataset.shuffle(images_list_len)

# Verify image paths were loaded and save one path for later in "some_image"
for f in full_dataset.take(5):
    some_image = f.numpy().decode("utf-8")
    print(f.numpy())
    
print("Some Image: ", some_image)

## Build an input pipeline

In [0]:

def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    return parts[-2] == parms.CLASS_NAMES

def decode_image(image):
    # convert the compressed string to a 3D uint8 tensor
    image = tf.image.decode_jpeg(image, channels=parms.IMAGE_CHANNELS)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    image = tf.image.convert_image_dtype(image, parms.IMAGE_DTYPE)
    # resize the image to the desired size.
    return tf.image.resize(image, [parms.IMAGE_ROWS, parms.IMAGE_COLS])

def image_aug(image):
    # do any augmentations
    if tf.random.uniform(()) > 0.25:    
        k = tf.random.uniform(shape=[], minval=1, maxval=4, dtype=tf.int32)
        image = tf.image.rot90(image, k) #0-4, 0/270, 90/180/270

    image = tf.clip_by_value(image, 0, 1)  # always clip back to 0, 1 before returning
    return image

def process_path_train(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    image = tf.io.read_file(file_path)
    image = decode_image(image)
    # add any augmentations
    image = image_aug(image)
    return image, label

def process_path_val(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    image = tf.io.read_file(file_path)
    image = decode_image(image)
    return image, label

In [0]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()
    ds = ds.batch(parms.BATCH_SIZE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

In [0]:
# display images....        
def show_batch(image_batch, label_batch, number_to_show=25):
    plt.figure(figsize=(10,10))
    show_number = number_to_show
    if parms.BATCH_SIZE < number_to_show:
        show_number = parms.BATCH_SIZE
        
    for n in range(show_number):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(tf.keras.preprocessing.image.array_to_img(image_batch[n]))
        plt.title(parms.CLASS_NAMES[np.argmax(label_batch[n])].title())
        plt.axis('off')

In [0]:
# split into training and validation sets of images
train_len = int(0.9 * images_list_len)
val_len = images_list_len - train_len

# Create datasets with new sizes
train_dataset = full_dataset.take(train_len)  #  Creates dataset with new size
val_dataset = full_dataset.skip(train_len)  # Creates dataset after skipping over the size

print("Total number: ", images_list_len, "  Train number: ", train_len, "  Val number: ", val_len)

In [0]:
# map training images to processing, includes any augmentation
train_dataset = train_dataset.map(process_path_train, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, label in train_dataset.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())
    
# Ready to be used for training
train_dataset = prepare_for_training(train_dataset)


In [0]:
# map validation images to processing
val_dataset = val_dataset.map(process_path_val, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, label in val_dataset.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())
    
# Ready to be used for training
val_dataset = prepare_for_training(val_dataset)



In [0]:
# Test Training

image_batch, label_batch = next(iter(train_dataset))
show_batch(image_batch.numpy(), label_batch.numpy())

In [0]:
# Test Validation

image_batch, label_batch = next(iter(val_dataset))
show_batch(image_batch.numpy(), label_batch.numpy())

## Build  model
- add and validate pretrained model as a baseline

In [0]:
# Create any call backs for training...These are the most common.

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

reduce_lr = ReduceLROnPlateau(monitor='loss', patience=2, verbose=1, min_lr=1e-6)
earlystopper = EarlyStopping(patience=8, verbose=1)
checkpointer = ModelCheckpoint(parms.MODEL_PATH, monitor='val_loss', verbose=1, mode="auto", save_best_only=True)
#csv_logger = CSVLogger(self.cvslogfile, append=True, separator=';')

#from keras.callbacks import TensorBoard
#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))


In [0]:
# Create model and compile it

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization, UpSampling2D, Conv2DTranspose, Concatenate, Activation
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, Adam, Nadam, SGD
########
#new with transfer learning
from tensorflow.keras.applications import MobileNet, imagenet_utils
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D

actual_MobileNet = tf.keras.applications.mobilenet.MobileNet()
        
def set_train_layers(model, train_layers=20): #since 224x224x3, set the first 20 layers of the network to be non-trainable
    if train_layers == 0: #set all non-trainable
        for layer in model.layers:
            layer.trainable=False
    else:
        for layer in model.layers[:train_layers]:             
            layer.trainable=False
        for layer in model.layers[train_layers:]:
            layer.trainable=True
    return model

def predict_image(image):     
    image = np.expand_dims(image, axis=0)
    image = tf.keras.applications.mobilenet.preprocess_input(image)
    predictions = actual_MobileNet.predict(image)
    results = imagenet_utils.decode_predictions(predictions)
    return results #list of decoded imagenet results


def build_model(CFG):
    base_model=MobileNet(weights='imagenet',include_top=False, input_shape=parms.IMAGE_DIM) #imports the mobilenet model and discards the last 1000 neuron layer.
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
    x=Dense(1024,activation='relu')(x) #dense layer 2
    x=Dense(512,activation='relu')(x) #dense layer 3
    preds=Dense(parms.NUM_CLASSES, activation=parms.FINAL_ACTIVATION)(x) #final layer
    model=Model(inputs=base_model.input,outputs=preds)
    return model

def compile_model(CFG, model):
    model.compile(loss=parms.LOSS,
          #optimizer=SGD(lr=0.001, momentum=0.9),
          optimizer=Adam(),
          metrics=parms.METRICS)
    return model


In [0]:
#test an image just using MobileNet
from tensorflow.keras.preprocessing import image
img = image.load_img(some_image, target_size=(224, 224))
img_array = image.img_to_array(img)
result = predict_image(img_array)
result

#str(parms.CLASS_NAMES[0])+'/*'))

In [0]:
#show the image...
from IPython.display import Image
Image(filename=some_image)

In [0]:
# Show the activation layers, can be trained or initial model (BETA)
#model_raw = build_model(CFG)
#img_path = os.path.join(parms.TRAIN_PATH, "Cat/2.jpg")
#image_show_seq_model_layers_BETA(img_path, model_raw, parms.IMAGE_DIM, 
#                                 activation_layer_num=0, activation_channel_num=11)


## Train model

In [0]:
# Train model
steps_per_epoch = np.ceil(train_len // parms.BATCH_SIZE) # set step sizes based on train & batch
validation_steps = np.ceil(val_len // parms.BATCH_SIZE) # set step sizes based on val & batch

model = build_model(parms)
model = compile_model(parms, model)

history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=parms.EPOCS, 
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    callbacks=[reduce_lr, earlystopper, checkpointer] # include any callbacks...
                    )


In [0]:
# Plot the training history
history_df = pd.DataFrame(history.history)
plt.figure()
history_df[['loss', 'val_loss']].plot(title="Loss")
plt.xlabel('Epocs')
plt.ylabel('Loss')
history_df[['accuracy', 'val_accuracy']].plot(title="Accuracy")
plt.xlabel('Epocs')
plt.ylabel('Accuracy')
plt.show()

## Validate model's predictions
- Create actual_lables and predict_labels
- Calculate Confusion Matrix & Accuracy
- Display results


In [0]:
#Load saved model
from tensorflow.keras.models import load_model 
def load_saved_model(model_path):
    model = load_model(model_path)
    print("loaded: ", model_path)
    return model

model = load_saved_model(parms.MODEL_PATH)

In [0]:
# Use model to generate predicted labels and probabilities
#labels, predict_labels, predict_probabilities, bad_results = predictions_using_dataset(model, val_dataset, 1, parms.BATCH_SIZE, create_bad_results_list=False)
labels, predict_labels, predict_probabilities, bad_results = predictions_using_dataset(model, val_dataset, validation_steps, parms.BATCH_SIZE, create_bad_results_list=False)


In [0]:
show_confusion_matrix(labels, predict_labels, parms.CLASS_NAMES)

In [0]:
# Graph the results

display_prediction_results(labels, predict_labels, predict_probabilities, parms.NUM_CLASSES, parms.CLASS_NAMES)


In [0]:
#Create a df from the bad results list, can save as csv or use for further analysis
bad_results_df = pd.DataFrame(bad_results, columns =['actual', 'predict', 'prob', 'image'])
bad_results_df.head()

In [0]:
# default is to not return bad_results, change to include them, create_bad_results_list=True

#bad_act, bad_pred, bad_prob, bad_images = zip(*bad_results)

In [0]:
# display bad images....        
def show_bad_batch(image_batch, bad_act, bad_pred, number_to_show=25):
    plt.figure(figsize=(10,10))
    show_number = number_to_show
    if len(image_batch) < number_to_show:
        show_number = len(image_batch)

    for n in range(show_number):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(tf.keras.preprocessing.image.array_to_img(image_batch[n][0]))
        #s = parms.CLASS_NAMES[bad_pred[n][0]]
        s = "Act: "+ str(bad_act[n][0]) + " Pred: " + str(bad_pred[n][0])
        plt.title(s)
        plt.axis('off')

In [0]:
print("    0)", parms.CLASS_NAMES[0],
      "  1)", parms.CLASS_NAMES[1],
      "  2)", parms.CLASS_NAMES[2],
      "  3)", parms.CLASS_NAMES[3],
      "  4)", parms.CLASS_NAMES[4])
#show_bad_batch(bad_images, bad_act, bad_pred)